In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=100, 
    zl_max=2.0, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 400.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 20000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [2]:
import h5py
import numpy as np 
from multiprocessing import Pool
import os

current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    fn = h5py.File(f"{current_dir}/samples/lenses_{count_draw}.hdf5", "r")
    src_z = fn['source/z'][()]
    src_Re = fn['source/Re'][()]
    src_q = fn['source/q'][()]
    src_pa = fn['source/pa'][()]
    src_app_mag_g = fn['source/app_mag_g'][()]
    src_app_mag_r = fn['source/app_mag_r'][()]
    src_app_mag_i = fn['source/app_mag_i'][()]
    src_thetaE = fn['source/thetaE'][()]

    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    mu_arr = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]

    fn.close()

    n_ideal_lens = src_thetaE.shape[1]

    if stack:
        #stack band
        this_obs_cond = obs_cond.astype('int')
        this_obs_cond = this_obs_cond[0, -1, :, :] #src-0, band-stack, nlens, 3-condition
        this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [nlens]
        bool_cond = (this_obs_cond==3)
    else:
        #individual band
        this_obs_cond = obs_cond.astype('int')
        this_obs_cond = this_obs_cond[:, :-1, :, :] #remove stack band
        cond = np.sum(this_obs_cond, axis=3)[0,:,:] #cond: [nband, nlens]
        bool_cond = (cond==3).astype('int') #cond: [nband, nlens]
        bool_cond = (np.sum(bool_cond, axis=0)>0) #bool_cond" [nlens]
        
#     print(SNR.shape)
#     print(SNR[0, :, bool_cond].shape)

    n_detect_lens = len(np.where(bool_cond)[0])
    return n_detect_lens, n_ideal_lens, SNR[0, :, bool_cond], mu_arr[0, -1, bool_cond], ring_cond[0, -1, bool_cond], obs_cond

n_detect_lens, n_ideal_lens, snr, mu, ring_cond, obs_cond = return_detected(0, stack=True)

In [3]:
pool = Pool(processes=24)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_detect_lens = [item[0] for item in results]
n_ideal_lens = [item[1] for item in results]
snr = [item[2] for item in results]
mu = [item[3] for item in results]
ring_cond = [item[4] for item in results]
obs_cond = [item[5] for item in results]

n_detect_lens = sum(n_detect_lens)
n_ideal_lens = sum(n_ideal_lens)
snr_stack_false = np.concatenate(snr, axis=0)
mu = np.concatenate(mu, axis=0)
ring_cond = np.concatenate(ring_cond, axis=0)
obs_cond = np.concatenate(obs_cond, axis=2)

etg_lensing_rate = n_ideal_lens / N_etgs
detect_rate_of_ideal_lens = n_detect_lens / n_ideal_lens
ring_lenses_rate = len(np.where(ring_cond)[0])/n_detect_lens
N_non_resolved_lenses = len(np.where(~obs_cond[0, -1, :, 0])[0])
N_non_tangent_arc_lenses = len(np.where(~obs_cond[0, -1, :, 1])[0])
N_lack_snr_lenses = len(np.where(~obs_cond[0, -1, :, 2])[0])

print('----------rate----------------')
print(f'ideal lensing rate of early type galaxy: {etg_lensing_rate:.2%}, == {n_detect_lens}/{N_etgs}')
print(f'ideal lenses detectable rate: {detect_rate_of_ideal_lens:.2%}')
print(f'rate of a lens is ring-like: {ring_lenses_rate:.2%}')
print(f'{(N_non_resolved_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *resolved image condition*')
print(f'{(N_non_tangent_arc_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *tangential arc condition*')
print(f'{(N_lack_snr_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *lacking enough SNR*')
print('------------------------------')
print()

from SimCsstLens.SimLensImage.MockSurvey import MockSurvey
survey = MockSurvey(config_path=f'.', config_file='csst_setting.yaml')
factor = survey.survey_sky_frac/sky_frac
observed_lens_survey = n_detect_lens*factor
print('----------lenses number----------------')
print("n lenses whole survey", observed_lens_survey)
print("n ring-like lenses whole survey", observed_lens_survey*ring_lenses_rate)
print('------------------------------')

----------rate----------------
ideal lensing rate of early type galaxy: 1.08%, == 11478/10739698.902726175
ideal lenses detectable rate: 9.93%
rate of a lens is ring-like: 49.76%
32.30% of ideal lenses are not detectable partially due to *resolved image condition*
67.03% of ideal lenses are not detectable partially due to *tangential arc condition*
75.70% of ideal lenses are not detectable partially due to *lacking enough SNR*
------------------------------

----------lenses number----------------
n lenses whole survey 11477.999652368362
n ring-like lenses whole survey 5711.999827001924
------------------------------


In [4]:
pool = Pool(processes=24)
vars = list(zip(list(range(Ndraw)), [False]*Ndraw))
results = pool.starmap(return_detected, vars)

n_detect_lens = [item[0] for item in results]
n_ideal_lens = [item[1] for item in results]
snr = [item[2] for item in results]
mu = [item[3] for item in results]
ring_cond = [item[4] for item in results]
obs_cond = [item[5] for item in results]

n_detect_lens = sum(n_detect_lens)
n_ideal_lens = sum(n_ideal_lens)
snr_stack_false = np.concatenate(snr, axis=0)
mu = np.concatenate(mu, axis=0)
ring_cond = np.concatenate(ring_cond, axis=0)
obs_cond = np.concatenate(obs_cond, axis=2)

etg_lensing_rate = n_ideal_lens / N_etgs
detect_rate_of_ideal_lens = n_detect_lens / n_ideal_lens
ring_lenses_rate = len(np.where(ring_cond)[0])/n_detect_lens
N_non_resolved_lenses = len(np.where(~obs_cond[0, -1, :, 0])[0])
N_non_tangent_arc_lenses = len(np.where(~obs_cond[0, -1, :, 1])[0])
N_lack_snr_lenses = len(np.where(~obs_cond[0, -1, :, 2])[0])

print('----------rate----------------')
print(f'ideal lensing rate of early type galaxy: {etg_lensing_rate:.2%}, == {n_detect_lens}/{N_etgs}')
print(f'ideal lenses detectable rate: {detect_rate_of_ideal_lens:.2%}')
print(f'rate of a lens is ring-like: {ring_lenses_rate:.2%}')
print(f'{(N_non_resolved_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *resolved image condition*')
print(f'{(N_non_tangent_arc_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *tangential arc condition*')
print(f'{(N_lack_snr_lenses/n_ideal_lens):.2%} of ideal lenses are not detectable partially due to *lacking enough SNR*')
print('------------------------------')
print()

from SimCsstLens.SimLensImage.MockSurvey import MockSurvey
survey = MockSurvey(config_path=f'.', config_file='csst_setting.yaml')
factor = survey.survey_sky_frac/sky_frac
observed_lens_survey = n_detect_lens*factor
print('----------lenses number----------------')
print("n lenses whole survey", observed_lens_survey)
print("n ring-like lenses whole survey", observed_lens_survey*ring_lenses_rate)
print('------------------------------')

----------rate----------------
ideal lensing rate of early type galaxy: 1.08%, == 7499/10739698.902726175
ideal lenses detectable rate: 6.49%
rate of a lens is ring-like: 51.21%
32.30% of ideal lenses are not detectable partially due to *resolved image condition*
67.03% of ideal lenses are not detectable partially due to *tangential arc condition*
75.70% of ideal lenses are not detectable partially due to *lacking enough SNR*
------------------------------

----------lenses number----------------
n lenses whole survey 7498.999772879452
n ring-like lenses whole survey 3839.999883698773
------------------------------
